In [1]:
def main_page(url,topic):
    # 모듈 import
    import requests
    from bs4 import BeautifulSoup as bs
    import re
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    # 정치면 메인 페이지 요청 -> html로 파싱
    global headers
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    req = requests.get(url, headers = headers)
    target = bs(req.content, 'html.parser')

    url_main = 'https://news.naver.com/'

    total = {}
    link = {}
    if topic == '정치':
        select = target.select('div.cluster_foot_inner > a')
    else:
        select = target.select('div.cluster_head_inner > a')
        
    for idx, tag in enumerate(select):

        each_url = url_main + tag['href'] # 각 헤드라인 서브 페이지 링크

        try: 
            news_num = int(re.match('[0-9]+', tag.text).group()) # 관련 기사 갯수
        except:
            news_num = np.nan

        # 헤드라인 서브 페이지 요청 -> html로 파싱
        sub_req = requests.get(each_url, headers = headers)
        page = bs(sub_req.content, 'html.parser')

        # 헤드라인 서브 페이지에서 가장 첫번째 기사 제목 (편의상 넣음 꼭 필요한 것은 아님)
        topic = page.select('div > ul > li > dl > dt')[1].text.strip() 

        # link 딕셔너리에 헤드라인 서브 페이지 링크, 관련 기사 건수, 첫번째 기사 제목  
        link[idx] = {'head_link':each_url, '관련기사':news_num, 'topic': topic} 

        # 헤드라인 서브 페이지 별로 언론사 리스트 추출
        np_list = page.select('span.writing')
        counter = {}
        for np_name in np_list:
            if np_name.text in ['조선일보', '동아일보', '경향신문', '한겨레', '한국일보', '중앙일보']:
                if np_name.text not in counter:
                    counter[np_name.text] = 0
                counter[np_name.text] += 1
            else:
                pass
        total[idx] = counter # key: index / value : 언론사 별 기사 

    df1 = pd.DataFrame.from_dict(link, orient = 'index') # 링크, 관련기사 수, topic 있는 dataframe
    df2 = pd.DataFrame.from_dict(total, orient = 'index') # 각 언론사별 기사 갯수 있는 dataframe

    # merge
    df3 = pd.merge(df1.reset_index(), df2.reset_index(), on = 'index', how = 'outer').drop('index', axis = 1)

    global con
    global pro
    con = ['조선일보', '동아일보']
    pro = ['경향신문', '한겨레']
    

    for name in con+pro:
        df3[f'{name}_presence'] = df3[f'{name}'].apply(lambda x :1 if x>0 else 0)
    df3['보수'] = df3['조선일보_presence'] + df3['동아일보_presence'] # 보수신문 2개 있는지
    df3['진보'] = df3['경향신문_presence'] + df3['한겨레_presence'] # 진보신문 2개 있는지
    df3['합계'] = df3['보수'] + df3['진보']
    df3 = df3.drop(['조선일보_presence','동아일보_presence','경향신문_presence','한겨레_presence'], axis=1)
    
    return df3

def choice_title(url,topic):
    df3 = main_page(url,topic)
    
    global max_idx
    global selected_url
    
    if df3['합계'].argmax() == df3['관련기사'].argmax():
        max_idx = df3['합계'].argmax()
        selected_url = df3['head_link'][max_idx]
    elif len(df3[(df3['진보'] > 0) & (df3['보수'] > 0)]) > 0:
        max_idx = df3['관련기사'].argmax()
        selected_url = df3['head_link'][max_idx]
    else:
        max_idx = df3['합계'].argmax()
        selected_url = df3['head_link'][max_idx]


    global new_con
    global new_pro
    
    if df3['합계'][max_idx] == 4:
        print('관련기사 건수', df3['관련기사'].max())
        print('기사 4개, 선택된 url', selected_url)

        # 기사 4개 일때 언론사는 원래 그대로 
        new_con = con
        new_pro = pro
        print('선택한 언론사', new_pro + new_con)

    # 보수, 진보 합쳐서 3개 있는 df - 관련기사 가장 많은 헤드라인의 링크
    elif df3['합계'][max_idx] == 3:
        print('관련기사 건수', df3['관련기사'].max())
        print('기사 3개, 선택된 url', selected_url)
        # 여기에 포함 안된 언론사
        absent_name = df3.loc[max_idx][pro+con].notna().idxmin()
        print('제외된 언론사', absent_name)

        # 기사 3개 일때 선택할 언론사
        if absent_name in pro:
            new_pro = list(set(pro) - {absent_name}) + ['한국일보']
            new_con = con
        else:
            new_pro = pro
            new_con = list(set(con) - {absent_name}) + ['중앙일보']
        print('선택한 언론사', new_pro + new_con)

    # 보수, 진보 각각 1개 있는 df - 관련기사 가장 많은 헤드라인의 링크
    elif (df3['합계'][max_idx] < 3) &( df3['합계'][max_idx]>0):
        print('관련기사 건수', df3['관련기사'].max())
        print('기사 2개, 선택된 url', selected_url)
        # 여기에 포함 안된 언론사
        absent_name_con = df3.loc[max_idx][con].notna().idxmin()
        absent_name_pro = df3.loc[max_idx][pro].notna().idxmin()
        print('제외된 언론사', absent_name_con, absent_name_pro)

        # 보수, 진보 각각 1개씩 있을 때 선택할 언론사
        new_pro = list(set(pro) - {absent_name_pro}) + ['한국일보']
        new_con = list(set(con) - {absent_name_con}) + ['중앙일보']
        print('선택한 언론사', new_pro + new_con)

    return selected_url
# 선택된 헤드라인 페이지에서 신문사별로 링크 따오기
def choice_link(url,topic):
    
    import requests
    from bs4 import BeautifulSoup as bs
    import re
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    selected_url = choice_title(url,topic)
    req = requests.get(selected_url, headers = headers)
    target = bs(req.content, 'html.parser')

    con_link1 = {} # 보수 언론사 1
    con_link2 = {} # 보수 언론사 2
    pro_link1 = {} # 진보 언론사 1
    pro_link2 = {} # 진보 언론사 2
    con_num1 = 0 
    con_num2 = 0
    pro_num1 = 0
    pro_num2 = 0

    for tag in target.select('div > ul > li > dl'):
        if tag.select_one('dd > span.writing').text == new_con[0]: # 보수 언론사 1 이름과 동일할 경우
            con_num1 += 1 # 기사 번호
            con_link1[con_num1] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_con[1]: # 보수 언론사 2 이름과 동일할 경우
            con_num2 += 1 # 기사 번호
            con_link2[con_num2] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_pro[0]: # 진보 언론사 1 이름과 동일할 경우
            pro_num1 += 1 # 기사 번호
            pro_link1[pro_num1] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        if tag.select_one('dd > span.writing').text == new_pro[1]: # 진보 언론사 2 이름과 동일할 경우
            pro_num2 += 1 # 기사 번호
            pro_link2[pro_num2] = tag.select_one('dt > a')['href'] # 기사 번호 당 링크
        else:
            pass

    total_links = {new_con[0]: con_link1, new_con[1]: con_link2, new_pro[0]: pro_link1, new_pro[1]: pro_link2}
        
    def compare_time(links):
        article_times = {} 
        for key, url in links.items():
            req = requests.get(url, headers = headers)
            target = bs(req.content, 'html.parser')
            time = target.select('span.t11')[-1].text # 기사 입력 시간 추출 (최초 작성 후 수정본이 있을때 수정된 시간으로 추출 / 만약 최초 작성 시간 기준을 하고 싶으면 인덱스[0]) 
            time = time.replace('오후', 'PM').replace('오전', 'AM') # 오후 -> PM, 오전 -> AM 변경
            time = datetime.strptime(time, '%Y.%m.%d. %p %I:%M') # datetime 으로 파싱
            article_times[key] = time # key: 원래 딕셔너리의 key 입력, value: 기사 입력 시간
        
        selected_article = links[max(article_times, key = article_times.get)] # article_times 딕셔너리의 value가 max인(가장 최근인) key로 con_link1 딕셔너리의 value(링크)찾기
        return selected_article

    final_links = {}
    for name, link in total_links.items():
        try:
            final_links[name] = compare_time(link)
        except:
            pass

    return final_links
  
def make_df(topic):
    
    import requests
    from bs4 import BeautifulSoup as bs
    import re
    import pandas as pd
    import numpy as np
    from datetime import datetime
    
    global url
    if topic == '경제':
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'
    elif topic == '정치':
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100'
    elif topic == '사회':
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=102'

    final_links = choice_link(url,topic)
    # 정치면 기사 dataframe
    time = []
    media = []
    head = []
    body = []

    for name, link in final_links.items():
        req = requests.get(link, headers=headers)
        soup = bs(req.content, 'html.parser')
        time.append(soup.select_one('span.t11').text) # time
        media.append(soup.select_one('div.article_header > div.press_logo > a > img')['title']) # media
        title = soup.find(id= 'articleTitle')
        try:
            head.append(title.text)
            text = soup.find(id = 'articleBodyContents')
            body.append(text.text.split('_flash_removeCallback() {}\n\n')[-1])
        except:
            continue

    politics_df = pd.DataFrame({'time':time,'media':media,'title':head,'document':body})
    return politics_df

# url 지정
make_df('경제')


관련기사 건수 57.0
기사 2개, 선택된 url https://news.naver.com//main/clusterArticles.naver?id=c_202111071110_00000169&mode=LSD&mid=shm&sid1=101&oid=214&aid=0001161265
제외된 언론사 조선일보 경향신문
선택한 언론사 ['한겨레', '한국일보', '동아일보', '중앙일보']


,time,media,title,document
0,2021.11.21. 오후 4:54,중앙일보,"대체 얼마나 나오길래…'역대급' 종부세 폭탄 날, 회견도 연다",\t\n\t 올해분 종합부동산세 납세 고지서가 22일부터 날아든다. 지난해보다 크...
1,2021.11.21. 오후 3:39,한겨레,올해 종부세 ‘세금폭탄’ 아니라 ‘이 빠진 호랑이’에 가깝다,국세청 22일 종부세 고지서 발송종부세 내일 고지한다는데…집부자는 공제·증여 ‘방어...
2,2021.11.21. 오후 5:31,한국일보,"종부세 대상자 10만명 증가, 세수는 4배 늘듯... 고지서 22일부터 발송",늘어난 세 부담에 역대급 증여 이어질 듯월세로 세금 내는 월세화 현상 가속화 전망도...
